In [6]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
import csv
import re
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np

import sys
sys.path.append("./missing_data")

from mpl_toolkits.basemap import Basemap
from build_dataset import get_files_from_folder, extract_dates_pattern_airmass_rgb_20200101_0000
from build_dataset import load_cyclones_track_noheader, compute_pixel_scale, inside_tile, calc_tile_offsets, create_tile
from build_dataset import split_into_tiles_subfolders_and_track_dates
from medicane_utils.geo_const import latcorners, loncorners, x_center, y_center, basemap_obj
from medicane_utils.load_files import load_all_images

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
Xmin, Ymin, px_scale_x, px_scale_y = compute_pixel_scale()
1/px_scale_y/1000 * 224, 1/px_scale_x/1000 * 224

(672.3640767222608, 672.0122685182674)

In [ ]:
# Percorso alla cartella dei frame
input_dir = "../fromgcloud"
output_dir = "../airmassRGB/supervised" 
#os.makedirs(output_dir, exist_ok=True)
sorted_metadata_files = load_all_images(input_dir)

117764 files loaded.


In [ ]:
f_sorted_metadata = sorted_metadata_files[:32]
#f_sorted_metadata

In [ ]:
num_total_files = len(f_sorted_metadata)
# quanti blocchi di 16 consecutivi
num_frames = 16
num_subfolders = num_total_files // num_frames

### Dataframe di tutte le tiles etichettate - check

In [2]:
df_data = pd.read_csv("all_data.csv", dtype={
        "path": 'string',
        "tile_offset_x": 'int16',
        "tile_offset_y": 'int16',
        "label": 'category',
        "lat": 'float32',
        "lon": 'float32',
        "x_pix": 'Int16',
        "y_pix": 'Int16',
        "name": 'string'
    }, parse_dates=['datetime'])
df_data.drop(columns="Unnamed: 0", inplace=True)

In [3]:
df_data

,path,datetime,tile_offset_x,tile_offset_y,label,lat,lon,x_pix,y_pix,name
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,0,NaN,NaN,<NA>,<NA>,[]
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,112,0,0,NaN,NaN,<NA>,<NA>,[]
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,224,0,0,NaN,NaN,<NA>,<NA>,[]
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,336,0,0,NaN,NaN,<NA>,<NA>,[]
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,448,0,0,NaN,NaN,<NA>,<NA>,[]
...,...,...,...,...,...,...,...,...,...,...
2355275,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,560,112,0,NaN,NaN,<NA>,<NA>,[]
2355276,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,672,112,0,NaN,NaN,<NA>,<NA>,[]
2355277,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,784,112,0,NaN,NaN,<NA>,<NA>,[]
2355278,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,896,112,0,NaN,NaN,<NA>,<NA>,[]


In [4]:
# separo gruppi temporali contigui
df_data = df_data.sort_values('datetime') 
# Calcola la differenza temporale rispetto alla riga precedente
df_data['delta'] = df_data['datetime'].diff()

# Definisci i punti di rottura: True se la differenza è maggiore della frequenza attesa
df_data['new_group'] = (df_data['delta'] > pd.Timedelta(minutes=60))  # puoi aumentare il margine se serve

# Crea gli ID di gruppo cumulando i True
df_data['gruppo'] = df_data['new_group'].cumsum()

gruppi_date = [g for _, g in df_data.groupby('gruppo')]

In [5]:
lista_date = []
lungh_gruppi = []
date_inizio_fine_gruppi = []
for df in gruppi_date:
    group_datetime = list(df.groupby("datetime"))
    for gd in group_datetime:        
        lista_date.append(len(gd[1]))
    lungh_gruppi.append(len(df)/len(gd[1])) # divido per le 20 tile di ciascun istante temporale
    start_date = df.datetime.iloc[0]
    end_date = df.datetime.iloc[-1]
    date_inizio_fine_gruppi.append((start_date, end_date))


In [25]:
np.array(lungh_gruppi)/288, set(lista_date) # abbiamo 20 tile per ogni data

(array([ 10.00347222,  10.00347222,   8.00347222,   8.00347222,
          4.98611111,   2.93402778,   5.37847222,  54.65625   ,
        277.73611111,   4.00347222,   5.37847222,   8.97916667,
          0.81597222,   8.02083333]),
 {20})

In [42]:
date_inizio_fine_gruppi

[(Timestamp('2011-11-01 00:00:00'), Timestamp('2011-11-11 00:00:00')),
 (Timestamp('2014-11-03 00:00:00'), Timestamp('2014-11-13 00:00:00')),
 (Timestamp('2016-10-26 00:00:00'), Timestamp('2016-11-03 00:00:00')),
 (Timestamp('2017-11-13 00:00:00'), Timestamp('2017-11-21 00:00:00')),
 (Timestamp('2018-09-25 00:00:00'), Timestamp('2018-09-29 23:35:00')),
 (Timestamp('2018-09-30 01:40:00'), Timestamp('2018-10-03 00:00:00')),
 (Timestamp('2020-01-01 00:00:00'), Timestamp('2020-01-06 09:00:00')),
 (Timestamp('2020-02-03 09:05:00'), Timestamp('2020-03-29 00:55:00')),
 (Timestamp('2020-03-29 02:00:00'), Timestamp('2020-12-31 23:55:00')),
 (Timestamp('2021-10-29 00:00:00'), Timestamp('2021-11-02 00:00:00')),
 (Timestamp('2021-11-04 00:00:00'), Timestamp('2021-11-09 09:00:00')),
 (Timestamp('2023-02-24 00:00:00'), Timestamp('2023-03-05 00:00:00')),
 (Timestamp('2023-09-03 00:00:00'), Timestamp('2023-09-03 19:30:00')),
 (Timestamp('2023-09-03 23:25:00'), Timestamp('2023-09-12 00:00:00'))]

### Sembra ben formato -> procedo a creare i video, per ogni blocco temporale

In [9]:
df = gruppi_date[0]

In [28]:
df

,path,datetime,tile_offset_x,tile_offset_y,label,lat,lon,x_pix,y_pix,name,delta,new_group,gruppo
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,0,NaN,NaN,<NA>,<NA>,[],NaT,False,0
19,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,1008,112,0,NaN,NaN,<NA>,<NA>,[],0 days,False,0
18,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,896,112,0,NaN,NaN,<NA>,<NA>,[],0 days,False,0
17,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,784,112,0,NaN,NaN,<NA>,<NA>,[],0 days,False,0
16,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,672,112,0,NaN,NaN,<NA>,<NA>,[],0 days,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57608,../fromgcloud/airmass_rgb_20111111_0000.png,2011-11-11,896,0,0,NaN,NaN,<NA>,<NA>,[],0 days,False,0
57604,../fromgcloud/airmass_rgb_20111111_0000.png,2011-11-11,448,0,0,NaN,NaN,<NA>,<NA>,[],0 days,False,0
57605,../fromgcloud/airmass_rgb_20111111_0000.png,2011-11-11,560,0,0,NaN,NaN,<NA>,<NA>,[],0 days,False,0
57606,../fromgcloud/airmass_rgb_20111111_0000.png,2011-11-11,672,0,0,NaN,NaN,<NA>,<NA>,[],0 days,False,0


In [7]:
grouped = df.groupby(["tile_offset_x", "tile_offset_y"], group_keys=False)

In [8]:
len(grouped), len(df)

(20, 57620)

In [16]:
df.datetime.iloc[0].strftime("%d-%m-%Y_%H%M")

'01-11-2011_0000'

In [ ]:
def create_tile_videos(df, output_dir=None):
    """
    df contiene:
      tile_offset_x, tile_offset_y, datetime, path, ...
    Ritorna una lista (videos_list) di DataFrame,
    ognuno con 16 righe consecutive. 
    """

    # 1) Ordiniamo il DataFrame per (tile_offset_x, tile_offset_y, datetime)
    df_sorted = df.sort_values(["tile_offset_x", "tile_offset_y", "datetime"])

    # 2) Raggruppiamo per tile_offset
    grouped = df_sorted.groupby(["tile_offset_x", "tile_offset_y"], group_keys=False)
    
    results = []
    video_id = 0
    
    for (tox, toy), group_df in grouped:
        # group_df è un sotto-DataFrame con tutte le righe di quella tile
        # Ordinate già per datetime.
        group_df = group_df.reset_index(drop=True)
        row_count = len(group_df)
        num_blocks = row_count // 16  # quante volte possiamo formare un blocco di 16
        
        # se row_count non è multiplo di 16, rimarranno righe extra che ignoriamo (oppure gestisci diversamente)
        #         
        for i in range(num_blocks):
            start_i = i * 16
            end_i   = start_i + 16
            block_df = group_df.iloc[start_i:end_i]
            start_time = group_df.datetime.iloc[start_i]
            end_time = group_df.datetime.iloc[end_i]
            
            # se output dir è passato: salva le tile video in nuove cartelle
            date_str = end_time.strftime("%d-%m-%Y_%H%M")
            path_name = f"{date_str}_{tox}_{toy}"
            subfolder = path_name
            
            if output_dir is not None:  
                subfolder = Path(output_dir) / path_name                              
                subfolder.mkdir(parents=True, exist_ok=True)

                for k, row_ in enumerate(block_df.itertuples(index=False)):
                    orig_path = row_.path  # colonna col path dell'immagine
                    new_name = subfolder / f"img_{k+1:05d}.png"

                    default_offsets_list = calc_tile_offsets() # valori di default   w, h, tile_size, stride)
                    frame_img = Image.open(orig_path)
                    tiles_this_frame = create_tile(frame_img, default_offsets_list)
                    for t in tiles_this_frame:
                        t.save(new_name)
    
            if (block_df['label'] == '1').any():
                label = 1
            else:
                label = 0
            # A questo punto block_df ha 16 righe consecutive
            results.append({
                "video_id": video_id,
                "tile_offset_x": tox,
                "tile_offset_y": toy,
                "path": str(subfolder),
                "label": label,
                "start_time": start_time,
                "end_time": end_time,                
                "orig_paths": block_df["path"].tolist(),
            })
            video_id += 1

    return pd.DataFrame(results)


In [56]:
df_res = create_tile_videos(df)

In [60]:
m_cicloni = df_res.label == 1
m_non_cicloni = df_res.label == 0

In [62]:
df_res[m_non_cicloni]

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time
0,0,0,0,01-11-2011_0120_0_0,0,2011-11-01 00:00:00,2011-11-01 01:20:00
1,1,0,0,01-11-2011_0240_0_0,0,2011-11-01 01:20:00,2011-11-01 02:40:00
2,2,0,0,01-11-2011_0400_0_0,0,2011-11-01 02:40:00,2011-11-01 04:00:00
3,3,0,0,01-11-2011_0520_0_0,0,2011-11-01 04:00:00,2011-11-01 05:20:00
4,4,0,0,01-11-2011_0640_0_0,0,2011-11-01 05:20:00,2011-11-01 06:40:00
...,...,...,...,...,...,...,...
3595,3595,1008,112,10-11-2011_1840_1008_112,0,2011-11-10 17:20:00,2011-11-10 18:40:00
3596,3596,1008,112,10-11-2011_2000_1008_112,0,2011-11-10 18:40:00,2011-11-10 20:00:00
3597,3597,1008,112,10-11-2011_2120_1008_112,0,2011-11-10 20:00:00,2011-11-10 21:20:00
3598,3598,1008,112,10-11-2011_2240_1008_112,0,2011-11-10 21:20:00,2011-11-10 22:40:00


In [104]:
df_out[df_out.label ==1]

,path,start,end,label,start_time,end_time
22395,../airmassRGB/supervised/part316_336_0,1,16,1,2014-11-10 11:55:00,2014-11-10 13:10:00
22396,../airmassRGB/supervised/part317_336_0,1,16,1,2014-11-10 13:15:00,2014-11-10 14:30:00
23716,../airmassRGB/supervised/part1637_336_0,1,16,1,2020-03-15 23:05:00,2020-03-16 00:20:00
23717,../airmassRGB/supervised/part1638_336_0,1,16,1,2020-03-16 00:25:00,2020-03-16 01:40:00
23718,../airmassRGB/supervised/part1639_336_0,1,16,1,2020-03-16 01:45:00,2020-03-16 03:00:00
...,...,...,...,...,...,...
139242,../airmassRGB/supervised/part6763_896_112,1,16,1,2020-12-25 23:05:00,2020-12-26 00:20:00
139243,../airmassRGB/supervised/part6764_896_112,1,16,1,2020-12-26 00:25:00,2020-12-26 01:40:00
139244,../airmassRGB/supervised/part6765_896_112,1,16,1,2020-12-26 01:45:00,2020-12-26 03:00:00
139245,../airmassRGB/supervised/part6766_896_112,1,16,1,2020-12-26 03:05:00,2020-12-26 04:20:00


In [84]:
df_out = df_out.rename(columns={'folder':'path'})

In [85]:
df_out.to_csv("./df_videotiles_date.csv", index=False)

### Guardo alcuni video da cancellare dal disco per risparmiare spazio

In [128]:
df_out[df_out['path'].str.contains(r"/part43\d+", regex=True)].label.sum()

0

In [129]:
df_filt = df_out[df_out['path'].apply(os.path.isdir)]
print(f"Numero di cartelle esistenti: {len(df_filt)}, righe totali precedenti: {df_out.shape[0]}")

Numero di cartelle esistenti: 126001, righe totali precedenti: 147200


In [130]:
df_filt[df_filt.label==1].shape[0], df_filt[df_filt.label==0].shape[0], round(df_filt.label.sum()/df_filt.shape[0], 3)

(1512, 124489, 0.012)

In [62]:
def balance_df_by_label(df, label_column="label", random_state=478562):
    """
    Bilancia un DataFrame mantenendo tutte le righe con label=1
    e selezionando (in modo random) lo stesso numero di righe con label=0.
    Restituisce un DataFrame mescolato con uguale quantità di 1 e 0.
    
    Parametri:
    - df: DataFrame originale
    - label_column: il nome della colonna che contiene la label (default: 'label')
    - random_state: per riproducibilità del campionamento casuale
    """
    # Seleziona tutte le righe con label=1
    df_pos = df[df[label_column] == 1]
    # Seleziona tutte le righe con label=0
    df_neg = df[df[label_column] == 0]

    # Numero di righe positive
    num_pos = len(df_pos)
    # Esegui un sample sulle righe negative pari a num_pos
    df_neg_sampled = df_neg.sample(n=num_pos, random_state=random_state)

    # Concatena e mescola
    df_balanced = pd.concat([df_pos, df_neg_sampled])
    df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)

    return df_balanced

In [131]:
df_balanced = balance_df_by_label(df_filt)
print(len(df_balanced), df_balanced['label'].value_counts())

3024 label
1    1512
0    1512
Name: count, dtype: int64


In [137]:
df_balanced


,path,start,end,label,start_time,end_time
0,../airmassRGB/supervised/part6762_784_112,1,16,1,2020-12-25 21:45:00,2020-12-25 23:00:00
1,../airmassRGB/supervised/part6102_784_112,1,16,1,2020-11-19 04:50:00,2020-11-19 06:05:00
2,../airmassRGB/supervised/part6101_784_112,1,16,1,2020-11-19 03:30:00,2020-11-19 04:45:00
3,../airmassRGB/supervised/part3246_560_0,1,16,1,2020-06-13 11:20:00,2020-06-13 12:35:00
4,../airmassRGB/supervised/part1925_336_112,1,16,0,2020-04-01 00:05:00,2020-04-01 01:20:00
...,...,...,...,...,...,...
3019,../airmassRGB/supervised/part1653_560_0,1,16,1,2020-03-16 20:25:00,2020-03-16 21:40:00
3020,../airmassRGB/supervised/part1993_896_112,1,16,0,2020-04-04 18:45:00,2020-04-04 20:00:00
3021,../airmassRGB/supervised/part3207_448_112,1,16,1,2020-06-11 07:20:00,2020-06-11 08:35:00
3022,../airmassRGB/supervised/part2788_336_112,1,16,0,2020-05-18 23:05:00,2020-05-19 00:20:00


In [135]:
l = df_balanced.shape[0]
#l = 100
tr_p = int(l * 0.7)
val_p = int(l * 0.9)
print(f"ranges train e validation:  {tr_p} - {val_p}")
df_train = df_balanced[:tr_p]
df_val = df_balanced[tr_p:val_p]
df_test = df_balanced[val_p:]

ranges train e validation:  2116 - 2721


In [136]:
out_csv_label = os.path.join("./", "train_supervised.csv")
df_train.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "val_supervised.csv")
df_val.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "test_supervised.csv")
df_test.to_csv(out_csv_label, index=False)

### copia le immagini in sottocartelle .../partN

In [27]:
from build_supervised_dataset_no_tiles import split_into_subfolders_and_track_dates

In [ ]:
#subfolders = split_into_subfolders(sorted_filenames, output_dir)
subfolder_info = split_into_subfolders_and_track_dates(sorted_filenames, output_dir)

In [ ]:
#subfolder_info

### Suddivide in train/test/val e scrive i CSV

In [28]:
output_dir

'E:/Medicanes_Data/airmassRGB'

In [30]:
from build_dataset import create_csv

In [33]:
create_csv(output_dir)

File CSV generati:
Train: E:/Medicanes_Data/airmassRGB\train.csv
Test: E:/Medicanes_Data/airmassRGB\test.csv
Validation: E:/Medicanes_Data/airmassRGB\val.csv


# Per il dataset supervisionato 
(senza tiles -> sbagliato perché ogni etichetta va associata alla singola tile)

In [ ]:
from build_supervised_dataset_no_tiles import create_supervised_csv_from_info, is_in_medicane
from medicane_utils.load_files import load_medicane_intervals

In [36]:
folder_root = output_dir      # cartella con subfolder part1, part2 ...
medicane_csv = "./medicane_validi.csv"    # le date dei medicane
out_csv = "./dataset.csv"
create_supervised_csv_from_info(subfolder_info, medicane_csv, out_csv)

Creato CSV supervisionato in: ./dataset.csv
